# Digital Mentor



In [ ]:
import torch
import api_utils
import utils
from messages_proceses import interaction
from openai import OpenAI
from setpathtomedia import seleccion
from labs import text_audio
from load_image import load_input
from env_fire import subir_firebase

In [ ]:
import os

# Cargar las variables de entorno
eleven_api_key = os.environ.get('ELEVEN_LABS_KEY')
openai_api_key = os.getenv("OPENAI_API_KEY")

# Verificar si las variables están definidas
if not eleven_api_key or not openai_api_key:
    print("Error: Las variables de entorno no están definidas.")
    exit(1)
openai_client = OpenAI(api_key=openai_api_key)
openai_model = "gpt-4" 

In [ ]:
input_video_path, presentation_video_path, goodbye_video_path, results_path = seleccion(personaje='Albert', verbose=True)

In [ ]:
selected_voice_id = text_audio(eleven_api_key,verbose = True)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Loading lip model
model = utils.load_lip_model(device=device)
frames,fps=load_input(input_video_path)

In [ ]:
import gradio as gr

def mostrar_video_con_texto(prompt: str, modelo_seleccionado: str) -> str:
    """
    Procesa el prompt y el modelo seleccionado para:
      1. Obtener la respuesta mediante la función `interaction`
      2. Subir el video a Firebase (o servicio similar) a través de `subir_firebase`
      3. Combinar el video y los textos en un bloque HTML
    """
    if not prompt:
        return "Por favor, completa ambos campos."

    # Obtener la respuesta del modelo seleccionado
    try:
        respuesta = interaction(
            prompt,
            modelo_seleccionado,
            selected_voice_id,
            input_video_path,
            openai_client,
            openai_model,
            eleven_api_key,
            results_path,
            device,
            model
        )
    except Exception as e:
        return "Error en la interacción: {}".format(e)

    # Subir el video y obtener la URL resultante
    try:
        URL_VIDEO = subir_firebase(results_path)
        print("URL del video:", URL_VIDEO)
    except Exception as e:
        return "Error al subir el video: {}".format(e)

    # Generar el HTML para el reproductor de video (como en el código anterior)
    reproductor_video = """<video width="640" height="480" controls autoplay>
                            <source src="{}" type="video/mp4">
                            Your browser does not support the video tag.
                          </video>""".format(URL_VIDEO)

    # Formatear el prompt y la respuesta para mostrarlos en columnas
    prompt_text = '<div style="float:left; padding-right:20px;">{}</div>'.format(prompt.replace("\n", "<br>"))
    respuesta_text = '<div style="float:right; padding-left:20px;">{}</div>'.format(respuesta.replace("\n", "<br>"))

    # Combinar el reproductor de video y los textos en un contenedor HTML
    contenido = '<div style="overflow:auto;">{}<br>{}<br>{}</div>'.format(reproductor_video, prompt_text, respuesta_text)
    
    return contenido

# Crear la interfaz de Gradio con los elementos de entrada y salida
interfaz = gr.Interface(
    fn=mostrar_video_con_texto,
    inputs=[
        gr.Textbox(label="Pregunta"),
        gr.Radio(["GPT API", "Llama 1B", "Llama 3B"], label="Selecciona el modelo de lenguaje")
    ],
    outputs="html",
    title="Mentores Digitales",
    flagging_mode="never"
)

interfaz.launch()
